# 朴素贝叶斯迷你项目
想象你的老板从一个信号很差的地方给你留了条消息。有几个词根本听不清。你想基于他以前给你留的一些消息的录音，填补这些剩下的词。为此，给定一些额外的消息信息，我们将使用贝叶斯公式来计算给定单词正好能填入留白处的概率。  
回想贝叶斯公式：  
$$P(A|B)=P(B|A)*P(A)/P(B)$$
或者在我们这个案例中：  
$$P(某个特定词|前后文的词)=P(前后文的词|某个特定的词)*P(某个特定的词)/P(前后文的词)$$

### 1. 最大可能性假设

在这个练习中我们要根据前面一个单词，找出哪个单词最有可能跟在它后面  

实现 NextWordProbability 使得你可以传入一段话，一个词，返回一个字典。这个字典的键（keys）是出现在这个词后面的词，每个键（key）的值（value）是跟在后面这个词出现的次数。  

你可以用 .split() 方法来把 sample_memo 这段话中的词用空格分割开来。

In [64]:
sample_memo = '''
Milt, we're gonna need to go ahead and move you downstairs into storage B. We have some new people coming in, and we need all the space we can get. So if you could just go ahead and pack up your stuff and move it down there, that would be terrific, OK?
Oh, and remember: next Friday... is Hawaiian shirt day. So, you know, if you want to, go ahead and wear a Hawaiian shirt and jeans.
Oh, oh, and I almost forgot. Ahh, I'm also gonna need you to go ahead and come in on Sunday, too...
Hello Peter, whats happening? Ummm, I'm gonna need you to go ahead and come in tomorrow. So if you could be here around 9 that would be great, mmmk... oh oh! and I almost forgot ahh, I'm also gonna need you to go ahead and come in on Sunday too, kay. We ahh lost some people this week and ah, we sorta need to play catch up.
'''
#
#   Maximum Likelihood Hypothesis
#
#
#   In this quiz we will find the maximum likelihood word based on the preceding word
#
#   Fill in the NextWordProbability procedure so that it takes in sample text and a word,
#   and returns a dictionary with keys the set of words that come after, whose values are
#   the number of times the key comes after that word.
#   
#   Just use .split() to split the sample_memo text into words separated by spaces.

def NextWordProbability(sampletext,word):
    sample_words=sampletext.split()
    times=CountWordTimes(sample_words,word)
    dic={}
    for index in range(len(sample_words)):
        if sample_words[index]==word and index < len(sample_words):
            key=sample_words[index+1]
            if key in dic.keys():
                dic[key]+=1
            else:
                dic[key]=1
    final={}
    for i in dic.keys():
        final[i]=float(dic[i])/times
    return final
    
def CountWordTimes(sample,word):
    time=0
    for item in sample:
        if item == word:
            time += 1
    return time

In [51]:
# 例子
dic1=NextWordProbability(sample_memo,'need')
print(dic1)
# 返回值最大对应的key
max(dic1.items(),key=lambda x:x[1])[0]
# 或者
max(dic1,key=dic1.get)

{'to': 0.3333333333333333, 'all': 0.16666666666666666, 'you': 0.5}


0.5

### 2. 最佳贝叶斯分类器
#### 2.1 小练习
在这个测验中，我们将根据一行中可能出现在第一个空格中的单词来计算第二个缺词的最佳标号。  
'for--- ---'单词For后缺失两个单词，已知：  
P('for this'|'for--- ---')=0.4;  
P('for that'|'for--- ---')=0.3;  
P('for those'|'for--- ---')=0.3;   
  
P('this time'|'this--- ---')=0.6,P('this job'|'this--- ---')=0.4;  
P('that job'|'that--- ---')=0.8,P('that time'|'that--- ---')=0.2;  
P('those items'|'those--- ---')=1.  
#### 问：你预测在第二个空格应该填什么词？概率是多少？
'time': 0.4*0.6+0.3*0.2=0.3;  
'job': 0.4*0.4+0.3*0.8=0.4;  
'items': 0.3*1=0.3;  
第二个空格应该填'job'，概率为0.4.
#### 2.2 实例编程

In [66]:
corrupted_memo = '''
Yeah, I'm gonna --- you to go ahead --- --- complain about this. Oh, and if you could --- --- and sit at the kids' table, that'd be --- 
'''
data_list = sample_memo.strip().split()
words_to_guess = ["ahead","could"]
def LaterWords(sample,word,distance):
    '''@param sample: a sample of text to draw from
    @param word: a word occuring before a corrupted sequence
    @param distance: how many words later to estimate (i.e. 1 for the next word, 2 for the word after that)
    @returns: a single word which is the most likely possibility
    '''
    
    # TODO: Given a word, collect the relative probabilities of possible following words
    # from @sample. You may want to import your code from the maximum likelihood exercise.
    newdic1=NextWordProbability(sample,word)
    dic={}
    for key1 in newdic1:
        newdic2=NextWordProbability(sample,key1)
        for key2 in newdic2:
            newdic2[key2]=newdic2[key2]*newdic1[key1]
            if key2 in dic:
                dic.setdefault(key2,dic[key2]+newdic2[key2])
            else:
                dic.setdefault(key2,newdic2[key2])
    ret=max(dic,key=dic.get) #等价于 max(dic1.items(),key=lambda x:x[1])[0] 
    # TODO: Repeat the above process--for each distance beyond 1, evaluate the words that
    # might come after each word, and combine them weighting by relative probability
    # into an estimate of what might appear next.
    
    return ret
   
print(LaterWords(sample_memo,"and",2))

in
